In [325]:
#Importing required libraries
import selenium
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [12]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe")

In [13]:
driver.get('https://www.amazon.in/')
driver.maximize_window()

In [14]:
#Searching headphones in the search bar and clicking the search button
search_bar=driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys("mobiles")

driver.find_element_by_id('nav-search-submit-button').click()

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

In [15]:
#Creating a for loop to append the urls of first 3 pages
URL=[]  
for page in range(0,3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        URL.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break   
        
#Taking the empty lists
brand_name=[]
product_name=[]
Rating=[]
no_of_ratings=[]
price=[]
returns=[]
delivery=[]
availability=[]
other_details=[]
product_url=[]
     
#We are running a for loop for the url as it changes for every page and we are extracting the product url
for i in URL:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the brand name
    try:
        brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
    except NoSuchElementException as e:
        brand_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds   
    
    #Extracting the product name
    try:
        string=' '  #Empty string
        for j in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+j  #Extracting the text and appending to empty string
        product_name.append(string)  #Appending the string having the extracted text
    except NoSuchElementException as e:
            product_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds     
    
    #Extracting the ratings
    try:
        Rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except NoSuchElementException as e:
        Rating.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the number of ratings
    try:
        no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except NoSuchElementException as e:
        no_of_ratings.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the price
    try:
        try:
            price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try:
                price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except:
                price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except NoSuchElementException as e:
        price.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the Return/Exchange
    try:
        returns.append(driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']").text)
    except NoSuchElementException as e:
        returns.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting Expected Delivery
    try:
        delivery.append(driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException as e:
        delivery.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the availability of headphones
    try:
        availability.append(driver.find_element_by_xpath("//div[@id='availability']").text)
    except NoSuchElementException as e:    
        availability.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the other details available
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")]
        other_details.append(details[0])
    except NoSuchElementException as e:     
        other_details.append("-")
    driver.implicitly_wait(4)       

In [16]:
#Checking the length of data extracted
print(len(brand_name),len(product_name),len(Rating),len(no_of_ratings),len(price),len(returns),len(delivery),
      len(availability),len(other_details),len(product_url))

58 58 58 58 58 58 58 58 58 58


In [17]:
#Creating a dataframe and checking the data extracted
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':product_name,'Rating':Rating,'No of Ratings':no_of_ratings,
                     'Price':price,'Return/Exchange':returns,'Expected Delivery':delivery,'Availability':availability,
                     'Other details':other_details,'Product URL':product_url})
amazon

,Brand Name,Name of Product,Rating,No of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,OPPO,"A74 5G (Fantastic Purple, 6GB RAM, 128GB Sto...",4.2 out of 5 stars,"3,173 ratings","₹17,990.00",7 Days Replacement,"Saturday, July 17",In stock.,OS Android 11.0---RAM 6 GB---Product Dimension...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Tecno,"Spark Go 2021 (Maldives Blue, 2GB RAM, 32GB ...",3.9 out of 5 stars,22 ratings,"₹7,299.00",7 Days Replacement,"Saturday, July 17",In stock.,"OS Android 10.0 (Go), Android 10.0 (Go Edition...",https://www.amazon.in/gp/slredirect/picassoRed...
2,TECNO,"Spark 7T(Jewel Blue, 4GB RAM, 64GB Storage) ...",4.1 out of 5 stars,686 ratings,"₹8,999.00",7 Days Replacement,"Saturday, July 17",In stock.,"OS HiOS 7.6, Android 11---RAM 64 GB---Product ...",https://www.amazon.in/TECNO-Spark-Storage-Batt...
3,Samsung,"Galaxy M11 (Metallic Blue, 4GB RAM, 64GB Sto...",4.1 out of 5 stars,"4,050 ratings","₹10,499.00",7 Days Replacement,"Monday, July 19",In stock.,OS Android---RAM 4 GB---Product Dimensions 16....,https://www.amazon.in/Samsung-Metallic-Storage...
4,Samsung,"Galaxy M31 (Ocean Blue, 8GB RAM, 128GB Stora...",4.3 out of 5 stars,"215,677 ratings","₹16,999.00",7 Days Replacement,July 18 - 30,"In stock on July 25, 2021.\nOrder it now.",OS Android---RAM 8 GB---Product Dimensions 15....,https://www.amazon.in/Samsung-Galaxy-Ocean-128...
5,Redmi,"Note 9 Pro (Interstellar Black, 4GB RAM, 64G...",4.3 out of 5 stars,"64,570 ratings","₹13,999.00",7 Days Replacement,"Saturday, July 17",In stock.,OS Android---RAM 4 GB---Product Dimensions 16....,https://www.amazon.in/Redmi-Note-Pro-Interstel...
6,Samsung,"Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)",4.3 out of 5 stars,"215,677 ratings","₹14,999.00",7 Days Replacement,Aug 9 - 11,"In stock on August 7, 2021.\nOrder it now.",OS Android---RAM 6 GB---Product Dimensions 15....,https://www.amazon.in/Samsung-Galaxy-Ocean-128...
7,Vivo,"Y15 (Burgundy Red, 4GB RAM, 64GB Storage)",4.2 out of 5 stars,"4,999 ratings","₹12,500.00",7 Days Replacement,-,In stock.,OS Android 9.0---RAM 4 GB---Product Dimensions...,https://www.amazon.in/Vivo-Burgundy-Storage-Ad...
8,Oppo,"A31 (Fantasy White, 6GB RAM, 128GB Storage) ...",4.2 out of 5 stars,"20,832 ratings","₹12,490.00",7 Days Replacement,"Saturday, July 17",In stock.,OS Android---RAM 6 GB---Product Dimensions 16....,https://www.amazon.in/Oppo-Fantasy-Storage-Add...
9,Redmi,"9 Power (Mighty Black, 6GB RAM, 128GB Storag...",4.2 out of 5 stars,"46,642 ratings","₹12,999.00",7 Days Replacement,July 17 - Aug 22,"In stock on August 17, 2021.\nOrder it now.",OS Android---RAM 6 GB---Product Dimensions 16....,https://www.amazon.in/Test-Exclusive_2020_1138...


In [18]:
#Saving the data into a csv file
amazon.to_csv("Amazon_Headphones.csv")

#Closing the driver
driver.close()

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [25]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe")  

In [26]:
#Getting the website to driver
url=('https://images.google.com/')
driver.get(url)
driver.maximize_window()

In [31]:
#Defining a function which accepts image names as arguments
def get_images(keywords):
    driver.get(url)
    
    #Searching the keywords in the search bar and clicking the search button
    search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
    search_bar.send_keys(keywords)
    button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    button.click()
    driver.implicitly_wait(10)  #Making the driver wait for 10 seconds
    
    #Taking 5 pages for each keyword and scraping the image data
    for page in range(0,5):
        driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy']").send_keys(Keys.PAGE_DOWN)
        time.sleep(5)   #Delaying the page wait time for 2 seconds
        
        #Iterating data for first 100 results of the keywords
        picture=[image for image in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy']")[:100]]
        image_url=[]   #Empty list
        print(len(picture))  
        
        #Iterating the tags for getting url link text of 100 results
        if(len(picture)==100):  #Checking if the length is equal to 100
            for i in picture:
                try:
                    i.click()
                    image_url.append(i.get_attribute('href'))  #Extracting the link texts
                except NoSuchElementException as e:
                    image_url.append("Link Not Available")  #Exception message if urls are not available
            return image_url
        else:
            print("Image not available")  #Message if length of data is not equal

In [32]:
#Sending the keywords for scraping the urls of images
Fruits_images=get_images('fruits')
Cars_images=get_images('cars')
machine_learning=get_images('Machine Learning')

48
Image not available
48
Image not available
48
Image not available
100
48
Image not available
48
Image not available
100
48
Image not available
48
Image not available
100


In [33]:
#Create a DataFrame for our data extracted
Image_URL=pd.DataFrame({})
Image_URL['Fruits']=Fruits_images
Image_URL['Cars']=Cars_images
Image_URL['Machine Learning']=machine_learning

#Checking the dataframe
Image_URL

,Fruits,Cars,Machine Learning
0,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
1,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
2,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
3,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
4,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
...,...,...,...
95,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
96,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
97,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
98,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...


In [34]:
#Closing the driver
driver.close()

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV. 

In [69]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe")  

In [70]:
#Getting the website to driver
driver.get('https://www.flipkart.com/')
driver.maximize_window()

In [72]:
#Searching smartphone in the search bar and clicking the search button
search_bar=driver.find_element_by_xpath("//input[@type='text']")
search_bar.send_keys('Oppo')  

driver.find_element_by_xpath("//button[@type='submit']").click()  #Clicking the search button

In [73]:
#Empty lists to store the scraped data
product_urls=[]
brand_name=[]
phone_name=[]
phone_color=[]
RAM=[]
ROM=[]
p_cam=[]
s_cam=[]
d_size=[]
d_resolution=[]
processor=[]
processor_core=[]
battery=[]
price=[]

#Defining the url link of each smartphone for scraping the specifications details
urls=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")

#Extracting the product urls
for i in urls:
    product_urls.append(i.get_attribute('href'))
    
for i in product_urls:
    driver.get(i)
    
    #Button for expanding the specifications
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    
    #Extracting the brand name
    try:
        brand=driver.find_element_by_xpath("//span[@class='B_NuCI']").text.split()
        brand_name.append(brand[0])
    except NoSuchElementException as e:
        brand_name.append('-')
        
    #Extracting the smartphone name
    try:
        phone=driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[3]/td[2]")
        phone_name.append(phone.text)
    except NoSuchElementException as e:
        phone_name.append('-')  
        
    #Extracting the phone color
    try:
        color=driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[4]/td[2]")
        phone_color.append(color.text)
    except NoSuchElementException as e:
        phone_color.append('-') 
        
    #Extracting RAM details
    try:
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        RAM.append('-')   
        
    #Extracting ROM details
    try:
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except NoSuchElementException as e:
        ROM.append('-')
        
    #Extracting primary camera details
    try:
        p_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        p_cam.append('-')
        
    #Extracting secondary camera details
    try:
        s_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except NoSuchElementException as e:
        s_cam.append('-')
        
    #Extracting display size details
    try:
        d_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except NoSuchElementException as e:
        d_size.append('-')
        
    #Extracting display resolution details
    try:
        d_resolution.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        d_resolution.append('-')
        
    #Extracting Processor details
    try:
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except NoSuchElementException as e:
        processor.append('-')
        
    #Extracting processor core details
    try:
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except NoSuchElementException as e: 
        processor_core.append('-')
        
    #Extracting battery details
    try:
        battery.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except NoSuchElementException as e:
        battery.append('-')
        
    #Extracting price details
    try:
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except NoSuchElementException as e:
        price.append('-')

In [74]:
#Create a DataFrame for our data
flipkart=pd.DataFrame({})
flipkart['Brand']=brand_name
flipkart['Phone Name']=phone_name
flipkart['Color']=phone_color
flipkart['RAM']=RAM
flipkart['Storage (ROM)']=ROM
flipkart['Primary Camera']=p_cam
flipkart['Secondary Camera']=s_cam
flipkart['Display Size']=d_size
flipkart['Display Resolution']=d_resolution
flipkart['Processor']=processor
flipkart['Processor Core']=processor_core
flipkart['Battery Capacity']=battery
flipkart['Price']=price
flipkart['Product URL']=product_urls

#Checking the dataframe
flipkart

,Brand,Phone Name,Color,RAM,Storage (ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Core,Battery Capacity,Price,Product URL
0,OPPO,A33,Moonlight Black,3 GB,32 GB,13MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh,"₹9,990",https://www.flipkart.com/oppo-a33-moonlight-bl...
1,OPPO,A53s 5G,Crystal Blue,6 GB,128 GB,13MP + 2MP + 2MP,8MP Front Camera,16.56 cm (6.52 inch),1600 x 720 Pixels,MediaTek Dimensity 700,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹15,990",https://www.flipkart.com/oppo-a53s-5g-crystal-...
2,OPPO,A53s 5G,Ink Black,6 GB,128 GB,13MP + 2MP + 2MP,8MP Front Camera,16.56 cm (6.52 inch),1600 x 720 Pixels,MediaTek Dimensity 700,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹15,990",https://www.flipkart.com/oppo-a53s-5g-ink-blac...
3,OPPO,A53,Moonlight Black,4 GB,64 GB,13MP + 2MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh,"₹11,990",https://www.flipkart.com/oppo-a53-moonlight-bl...
4,OPPO,A53,Mint Cream,4 GB,64 GB,13MP + 2MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh,"₹11,990",https://www.flipkart.com/oppo-a53-mint-cream-6...
5,OPPO,A53,Mint Cream,6 GB,128 GB,13MP + 2MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh,"₹13,990",https://www.flipkart.com/oppo-a53-mint-cream-1...
6,OPPO,A33,Mint Cream,3 GB,32 GB,13MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh,"₹9,990",https://www.flipkart.com/oppo-a33-mint-cream-3...
7,OPPO,A53,Moonlight Black,6 GB,128 GB,13MP + 2MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 460,Octa Core,5000 mAh,"₹13,990",https://www.flipkart.com/oppo-a53-moonlight-bl...
8,OPPO,A12,Flowing Silver,4 GB,64 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹9,990",https://www.flipkart.com/oppo-a12-flowing-silv...
9,OPPO,A12,Black,3 GB,32 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹8,990",https://www.flipkart.com/oppo-a12-black-32-gb/...


In [75]:
#Saving the data into a csv file
flipkart.to_csv("Flipkart_OppoPhones.csv")

#Closing the driver
driver.close()

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [59]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe")  


In [60]:
#Getting the website to driver
driver.get('https://www.google.com/maps')
driver.maximize_window()
#When we run this line, automatically the webpage will be opened

In [61]:
#Creating user input for entering the city manually
city=input("Enter City: ")

#Finding the required element and sending the keys
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").send_keys(city)

Enter City: Bangalore


In [62]:
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").send_keys(Keys.ENTER)

In [63]:
coordinates=driver.current_url
lat=coordinates.split('@')[1].split(',')[0]
long=coordinates.split('@')[1].split(',')[1].split(',')[0]
print('Latitude: ',lat)
print('Longitude: ',long)

Latitude:  28.639232
Longitude:  77.348864


In [64]:
#Closing the driver
driver.close()

# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [76]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe")  


In [77]:
driver.get('https://trak.in/')
driver.maximize_window()

In [78]:
#Getting the Funding Deals details by fetching the link
funding_deals=driver.find_element_by_xpath("//li[@id='menu-item-51510']/a").get_attribute('href')
driver.get(funding_deals)

In [79]:
#Taking the empty lists of the details to be scraped
Date=[]
Startup=[]
Industry=[]
SubVertical=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

#For quarter July 20 - September 20, the range numbers are between 48-51. So we will consider that range and scrap data
for i in range(48,51):
    driver.find_element_by_xpath("//div[@id='tablepress-{}_wrapper']/div/label/select/option[4]".format(i)).click()
#As the tablepress value differs for each quarter, we are iterating in the for loop and converting into raw string using format
    
    #Scrapping date data from the webpage
    date=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[2]".format(i))
    for d in date:
        Date.append(d.text)
        
    #Scrapping startup details data from the webpage
    startup=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[3]".format(i))
    for s in startup:
        Startup.append(s.text)
        
    #Scrapping Industry details data from the webpage
    industry=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[4]".format(i))
    for ind in industry:
        Industry.append(ind.text)
        
    #Scrapping Subvertical details data from the webpage
    sv=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[5]".format(i))
    for s in sv:
        SubVertical.append(s.text)
        
    #Scrapping Location details data from the webpage
    location=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[6]".format(i))
    for l in location:
        Location.append(l.text)
        
    #Scrapping Investor details data from the webpage
    investor=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[7]".format(i))
    for inv in investor:
        Investor.append(inv.text)
        
    #Scrapping Investment type details data from the webpage
    investment=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[8]".format(i))
    for it in investment:
        Investment.append(it.text)
        
    #Scrapping the amount data from the webpage
    amount=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[9]".format(i))
    for a in amount:
        Amount.append(a.text)

In [80]:
#Creating a dataframe for saving the extracted data
trak_fund=pd.DataFrame({})
trak_fund['Date']=Date
trak_fund['Startup']=Startup 
trak_fund['Industry']=Industry
trak_fund['SubVertical']=SubVertical
trak_fund['Location']=Location
trak_fund['Investor']=Investor
trak_fund['Investment']=Investment
trak_fund['Amount']=Amount

#Checking the dataframe
trak_fund

,Date,Startup,Industry,SubVertical,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [81]:
#Closing the driver
driver.close()

# 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [82]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe")  

In [83]:
#Getting the website to driver
driver.get('https://www.digit.in/')
driver.maximize_window()

In [85]:
#Clicking on top 10 option
top10=driver.find_element_by_xpath("//div[@class='menu']/ul/li[4]/a")
top10.click()

#Clicking on best gaming laptops option from top10
best_gl=driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[26]/a")
best_gl.click()

In [86]:
#Extracting the tags having the laptop name
name=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")

#Extracting the text from the tags
prod_name=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in name:
    prod_name.append(i.text)

#Extracting the tags having the OS type
OS_type=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")

#Extracting the text from the tags
OS=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in OS_type:
    OS.append(i.text)

#Extracting the tags having display details
display=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")

#Extracting the text from the tags
display_specs=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in display:
    display_specs.append(i.text)

#Extracting the tags having processor details
processor=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")

#Extracting the text from the tags
processor_specs=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in processor:
    processor_specs.append(i.text)

#Extracting the tags having memory specs

#List of specification names
memory=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")

#Value of the specifications
memory_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")

#Now we will separate HDD and RAM text from memory specs tags
HDD=[]
RAM=[]  #Empty lists
for i in range(len(memory)):
    if memory[i].text=='Memory':
        HDD.append(memory_specs[i].text.split('/')[0])
        RAM.append(memory_specs[i].text.split('/')[1])
    else:
        HDD.append('Not Available')
        RAM.append('Not Available')

#Extracting the tags having weight

#List of specifications name
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
weights_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate weight text from tags
weight=[] #Empty list
for i in range(len(weights)):
    if weights[i].text=='Weight':
        weight.append(weights_specs[i].text)

#Extracting the tags having dimensions

#List of specifications name
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
dims_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate dimensions text from tags
dimension=[] #Empty list
for i in range(len(dims)):
    if dims[i].text=='Dimension':
        dimension.append(dims_specs[i].text)

#Extracting the tags having Graphics Processor

#List of specifications name
GPs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
GPs_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate GPs text from tags
GPU=[] #Empty list
for i in range(len(GPs)):
    if GPs[i].text=='Graphics Processor':
        GPU.append(GPs_specs[i].text)
        
#Extracting the tags having the price

#As there are some prices in the main url, we will go to full specs and scrape the prices
#First we will extract the urls of all laptop's full specs
full_specs=[]   #Empty list
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")

#Running a for loop for extraction of text from tags
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
        
#Now we will extract price by iterating full_specs
Price=[]  #Empty list
for i in full_specs:
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")  #Getting price tags
        Price.append(prices.text)  #Extracting text
    except NoSuchElementException as e:   #Running an exception if the price is not available
         Price.append("Not Available")    #Message to be printed in places where the price is not available            

In [87]:
#Creating a dataframe for saving our extracted data
laptops=pd.DataFrame({'Product Name':prod_name,'OS':OS,'Display':display_specs,'Processor':processor_specs,'HDD':HDD,'RAM':RAM,
                     'Weight':weight,'Dimension':dimension,'Graphic Processor':GPU,'Price':Price})
laptops

,Product Name,OS,Display,Processor,HDD,RAM,Weight,Dimension,Graphic Processor,Price
0,ALIENWARE AREA 51M R2,WINDOWS 10 HOME,"17.3"" (1920 X 1080)",10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ,1 TB SSD,16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,Not Available
1,ALIENWARE M15 R3,WINDOWS 10 HOME,"15.6"" (3840 X 2160)",10TH GENERATION INTEL® CORE™ I9-10980HK | NA,1 TB SSD,16 GBGB DDR4,NA,NA,NA,"341,990"
2,ASUS ROG STRIX SCAR 15,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN™ 9 5900HX | 3.3 GHZ,1 TB SSD,16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,Not Available
3,ASUS ROG ZEPHYRUS G14,WINDOWS 10 HOME,"14"" (1920 X 1080)",AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1 TB SSD,16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"164,990"
4,LENOVO LEGION 5I,WINDOWS 10 PRO,"15.6"" (1920 X 1080)",10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ,1 TB SSD,16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"71,990"
5,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10,"15.6"" (3840 X 1100)",INTEL CORE I7 10TH GEN 10875H | NA,512 GB SSD,4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"185,000"
6,ACER ASPIRE 7 GAMING,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN™ 5-5500U HEXA-CORE | NA,512 GB SSD,8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"56,990"


In [88]:
#Closing the driver
driver.close()

#8. Write a python program to scrape the details for all billionaires from www.forbes.com.
Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”,
“Industry”.

In [102]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe")  

In [104]:
#Getting the website to driver
driver.get('https://www.forbes.com/real-time-billionaires/#48e869f63d78')
driver.maximize_window()

In [133]:
Rank = []
Name = []
Net_Worth = []
Age = []
Citizenship = []
Source = []

size =  len(driver.find_elements_by_xpath("//*[@id='row-3']/div[2]/div/div[4]/div[1]/table/tbody/tr") )
print(size)

for i in range(1,size):
    if (i==11 or i==22):
        continue
    rank = driver.find_element_by_xpath("//*[@id='row-3']/div[2]/div/div[4]/div[1]/table/tbody/tr[%d]/td[2]" %(i)).text
    Rank.append(rank)

print(Rank)

for i in range(1,size):
    if (i==11 or i==22):
        continue
    name = driver.find_element_by_xpath("//*[@id='row-3']/div[2]/div/div[4]/div[1]/table/tbody/tr[%d]/td[3]" %(i)).text
    Name.append(name)

print(Name)

for i in range(1,size):
    if (i==11 or i==22):
        continue
    networth = driver.find_element_by_xpath("//*[@id='row-3']/div[2]/div/div[4]/div[1]/table/tbody/tr[%d]/td[4]" %(i)).text
    Net_Worth.append(networth)
    
print(Net_Worth)

for i in range(1,size):
    if (i==11 or i==22):
        continue
    age = driver.find_element_by_xpath("//*[@id='row-3']/div[2]/div/div[4]/div[1]/table/tbody/tr[%d]/td[6]" %(i)).text
    Age.append(age)
    
print(Age)

for i in range(1,size):
    if (i==11 or i==22):
        continue
    country = driver.find_element_by_xpath("//*[@id='row-3']/div[2]/div/div[4]/div[1]/table/tbody/tr[%d]/td[8]" %(i)).text
    Citizenship.append(country)
    
print(Citizenship)

for i in range(1,size):
    if (i==11 or i==22):
        continue
    source = driver.find_element_by_xpath("//*[@id='row-3']/div[2]/div/div[4]/div[1]/table/tbody/tr[%d]/td[7]" %(i)).text
    Source.append(source)
    
print(Source)
    

27
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
['Jeff Bezos', 'Bernard Arnault & family', 'Elon Musk', 'Bill Gates', 'Mark Zuckerberg', 'Larry Ellison', 'Larry Page', 'Sergey Brin', 'Warren Buffett', 'Francoise Bettencourt Meyers & family', 'Steve Ballmer', 'Mukesh Ambani', 'Amancio Ortega', 'Zhong Shanshan', 'Carlos Slim Helu & family', 'Jim Walton', 'Alice Walton', 'Rob Walton', 'MacKenzie Scott', 'Michael Bloomberg', 'Phil Knight & family', 'François Pinault & family', 'Gautam Adani & family', 'Ma Huateng']
['$210.5 B', '$189.8 B', '$162.1 B', '$129.5 B', '$125.2 B', '$113.4 B', '$111.4 B', '$107.9 B', '$100.9 B', '$87.8 B', '$83.0 B', '$79.2 B', '$78.2 B', '$76.1 B', '$70.6 B', '$64.7 B', '$64.3 B', '$63.9 B', '$61.9 B', '$59.0 B', '$58.6 B', '$55.5 B', '$55.5 B', '$55.4 B']
['57', '72', '50', '65', '37', '76', '48', '47', '90', '68', '65', '64', '85', '66', '81', '73', '71', '76', '51', '79

In [134]:
#Creating a dataframe for saving our extracted data
forbes=pd.DataFrame({'Rank':Rank,'Name':Name,'Net_Worth':Net_Worth,'Age':Age,'Citizenship':Citizenship,'Source':Source})
forbes

,Rank,Name,Net_Worth,Age,Citizenship,Source
0,1,Jeff Bezos,$210.5 B,57,United States,Amazon
1,2,Bernard Arnault & family,$189.8 B,72,France,LVMH
2,3,Elon Musk,$162.1 B,50,United States,"Tesla, SpaceX"
3,4,Bill Gates,$129.5 B,65,United States,Microsoft
4,5,Mark Zuckerberg,$125.2 B,37,United States,Facebook
5,6,Larry Ellison,$113.4 B,76,United States,software
6,7,Larry Page,$111.4 B,48,United States,Google
7,8,Sergey Brin,$107.9 B,47,United States,Google
8,9,Warren Buffett,$100.9 B,90,United States,Berkshire Hathaway
9,10,Francoise Bettencourt Meyers & family,$87.8 B,68,France,L'Oréal


In [135]:
#Saving the data into a csv file
forbes.to_csv("Forbes_List.csv")

#Closing the driver
driver.close()

#9. Write a program to extract at least 500 Comments, Comment upvote and time when comment
was posted from any YouTube Video.

In [179]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe") 

In [180]:
#Getting the website to driver
driver.get('https://www.youtube.com/watch?v=k2P_pHQDlp0')
driver.maximize_window()

In [193]:
Comments = []
Time = []
Upvote = []

comments = driver.find_elements_by_xpath("//*[@id='content-text']")
comments

[<selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="c9617845-0ac2-4039-a00e-fa51bfcd491e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="a60308dd-8408-48cd-a3cd-d42aabdeaa15")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="437d5141-6929-4f56-a21b-63a0ec097c71")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="b4caae90-9f1f-4363-ac3c-6b273c1ae520")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="1af4cf79-a0e4-421f-88de-ca81cda0970d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="2c63f22c-de42-4c54-af9e-61a2346d427d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="b5ad4e79-dc05-48aa-b24f-63

In [194]:
for x in comments[:500]:
    Comments.append(x.text)
    
Comments

['Please do subscribe my vlogging channel for motivational videos\nhttps://www.youtube.com/channel/UCjWY5hREA6FFYrthD0rZNIw',
 "I don't know why all the youtubers explains stuffs better than my teachers.\nBut this video is crystal clear to understand",
 "When I'm going to get a job on AI, I'm gonna donate my first salary to Mr.Krish Naik as a sign of respect.",
 "Finally a good explanation that doesn't hide confusion behind jargon. Thanks!",
 'currently watching as the quarantine period is in process and this helped me understand. Thank you for this amazing explanation! <3 <3',
 'Amazing content and a great way of teaching. Sir, I really liked your way of teaching. In fact, I am also trying to adapt your way of teaching.',
 "Correct. I've soon got my masters in IT engineering and I've taken courses on advanced level in AI and ML. The information in this video is accurate!",
 'Thank you for clearing all points.',
 'Perfect explanation, you have made my day',
 'I am proud be a your stude

In [195]:
time = driver.find_elements_by_xpath("//*[@id='header-author']/yt-formatted-string/a")
time

[<selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="20734eff-d3db-4cdd-84bc-e7ad3577e475")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="d5da4559-214f-4e96-a807-655aed6c0f66")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="2164c82f-bcd2-4e4f-ba99-849b7aa7fc85")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="616d0925-ba1a-4e42-8700-84b1e9bc4623")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="ee199580-b1ea-46dd-8a5f-9623de472ede")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="7d89f946-55d9-4b9c-bb2a-a833e21216df")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="8476d174-68e3-4bb2-9add-47

In [196]:
for x in time[:500]:
    Time.append(x.text)
    
Time

['3 months ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 month ago',
 '1 year ago',
 '8 months ago (edited)',
 '1 year ago',
 '1 year ago',
 '3 months ago',
 '4 months ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '4 months ago (edited)',
 '1 week ago',
 '1 year ago',
 '1 year ago',
 '5 months ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago (edited)',
 '10 months ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '10 months ago',
 '1 year ago',
 '1 year ago',
 '11 months ago',
 '1 year ago',
 '1 year ago',
 '1 hour ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '11 months ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '1 year ago',
 '9 months ago',
 

In [197]:
upvote = driver.find_elements_by_xpath("//*[@id='vote-count-middle']")
upvote

[<selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="bd72d651-c551-4e7c-b000-7bbc9d395e3e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="6bb3e8a4-ac59-4672-b034-eb6af35ff38d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="ea10fc22-acc1-4f16-b1ac-50875120d60c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="fc68c316-24ab-4db4-8627-0f1b954a01af")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="9a18f621-2db5-49b8-b6c5-17d6947699ba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="2c5306d5-c6d2-43dd-b7da-ec11dd6fd2f6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1af7c189835e1e0309c8156c6d6ec608", element="c1e8c674-80b6-456a-8349-8a

In [199]:
for x in upvote[:500]:
    Upvote.append(x.text)

Upvote

['16',
 '1.1K',
 '298',
 '328',
 '56',
 '23',
 '53',
 '21',
 '48',
 '2',
 '10',
 '6',
 '',
 '18',
 '2',
 '4',
 '16',
 '2',
 '2',
 '2',
 '4',
 '1',
 '1',
 '',
 '',
 '5',
 '2',
 '1',
 '1',
 '2',
 '5',
 '1',
 '1',
 '1',
 '',
 '3',
 '1',
 '2',
 '1',
 '1',
 '6',
 '1',
 '2',
 '1',
 '1',
 '3',
 '1',
 '2',
 '1',
 '',
 '4',
 '1',
 '2',
 '1',
 '1',
 '3',
 '',
 '1',
 '1',
 '',
 '3',
 '1',
 '',
 '',
 '1',
 '3',
 '2',
 '1',
 '',
 '',
 '3',
 '2',
 '1',
 '',
 '',
 '5',
 '1',
 '',
 '2',
 '1',
 '4',
 '1',
 '1',
 '1',
 '',
 '25',
 '',
 '1',
 '1',
 '',
 '5',
 '2',
 '',
 '1',
 '1',
 '3',
 '',
 '1',
 '1',
 '',
 '3',
 '',
 '',
 '',
 '2',
 '6',
 '',
 '',
 '',
 '',
 '25',
 '1',
 '',
 '1',
 '1',
 '8',
 '1',
 '1',
 '1',
 '1',
 '22',
 '',
 '',
 '',
 '1',
 '4',
 '',
 '1',
 '1',
 '1',
 '22',
 '',
 '',
 '',
 '1',
 '3',
 '',
 '2',
 '',
 '1',
 '17',
 '1',
 '',
 '1',
 '1',
 '10',
 '',
 '',
 '',
 '1',
 '3',
 '',
 '1',
 '1',
 '',
 '7',
 '',
 '',
 '1',
 '',
 '3',
 '',
 '',
 '',
 '1',
 '12',
 '',
 '1',
 '2',
 '1',
 '8',
 

In [200]:
#Checking the length of data extracted
print(len(Comments),len(Time),len(Upvote))

340 340 340


In [201]:
#Creating a dataframe for saving our extracted data
Youtube=pd.DataFrame({'Comments':Comments,'Time':Time,'Comments Upvote':Upvote})
Youtube

,Comments,Time,Comments Upvote
0,Please do subscribe my vlogging channel for mo...,3 months ago,16
1,I don't know why all the youtubers explains st...,1 year ago,1.1K
2,"When I'm going to get a job on AI, I'm gonna d...",1 year ago,298
3,Finally a good explanation that doesn't hide c...,1 year ago,328
4,currently watching as the quarantine period is...,1 year ago,56
...,...,...,...
335,Facebook photo tagging sir one of that example,1 year ago,7
336,Sir I am going to take admission in computer s...,2 weeks ago,
337,"Really nice video, helpful, explanatory, clear...",1 year ago,
338,Simple and clear. Very well explained. 👍,1 year ago,


In [202]:
#Saving the data into a csv file
Youtube.to_csv("Youtube_Comments.csv")

#Closing the driver
driver.close()

#10. Write a python program to scrape a data for all available Hostels from
https://www.hostelworld.com/ in “London” location. You have to scrape hostel name,
distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms

In [289]:
#Connect to web driver
driver=webdriver.Chrome(r"C:\\Users\\Admin\\Downloads\\chromedriver.exe") 

In [290]:
#Getting the website to driver
driver.get('https://www.hostelworld.com')
driver.maximize_window()

In [293]:
#Searching London in the search bar and clicking the search button
search_bar=driver.find_element_by_id('location-text-input-field')
search_bar.send_keys("London")
search_bar.find_element_by_xpath("//*[@id='predicted-search-results']/li[2]").click()

driver.find_element_by_id('search-button').click()

In [294]:
hostel_name = driver.find_elements_by_xpath("//*[@class='title title-6']")

In [297]:
Hostel_Name = []
for i in hostel_name:
    Hostel_Name.append(i.text)
    
Hostel_Name

['Generator London',
 "St Christopher's Village",
 'Safestay London Kensington Holland Park',
 'PubLove @ The Crown, Battersea',
 'The London Home Hostel',
 '247london Hostel and Private Rooms',
 'PubLove @ The White Ferry, Victoria',
 'Prime Backpackers Angel',
 'The Dictionary Hostel',
 'Astor Museum Inn',
 "Wombat's The City Hostel London",
 'Safestay London Elephant & Castle',
 'PubLove @ The Green Man, Paddington',
 'No.8 Willesden Hostel London',
 'PubLove @ The Steam Engine, Waterloo',
 'Atlas Hostels',
 'SoHostel',
 'Keystone House Kings Cross',
 'Jaeger Acton Hostel',
 'Abercorn House',
 'Astor Hyde Park',
 'Phoenix Hostel',
 'Clink261',
 'Urbany Hostel London',
 'New Cross Inn Hostel',
 'Smart Hyde Park Inn Hostel',
 'PubLove @ The Exmouth Arms, Euston',
 'Astor Kensington',
 'No.8 Seven Sisters',
 "St Christopher's Oasis-Female Only"]

In [298]:
distance = driver.find_elements_by_xpath("//*[@class='description']")

In [299]:
Distance = []
for i in distance:
    Distance.append(i.text)

Distance

['Hostel - 3km from city centre',
 'Hostel - 1.8km from city centre',
 'Hostel - 5.9km from city centre',
 'Hostel - 4.7km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 8.2km from city centre',
 'Hostel - 2.4km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 4.1km from city centre',
 'Hostel - 2.2km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 1.7km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 10km from city centre',
 'Hostel - 0.5km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 2km from city centre',
 'Hostel - 3.5km from city centre',
 'Hostel - 10.4km from city centre',
 'Hostel - 7.3km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 4.2km from city centre',
 'Hostel - 3.2km from city centre',
 'Hostel - 5.4km from city centre',
 'Hostel - 6.1km from city centre',
 'Hostel - 5km from city centre',
 'Hostel - 3.4km from city centre',
 'Hostel - 4.9km from city centre'

In [300]:
rating = driver.find_elements_by_xpath("//*[@class='rating rating-summary-container big']")

In [301]:
Rating = []
Overall_Reviews = []
Total_Reviews = []
for i in rating:
    i = i.text
    i = i.split("\n")
    Rating.append(i[0])

Rating

['7.5',
 '8.8',
 '9.6',
 '5.6',
 '7.7',
 'No Rating',
 '8.9',
 '10',
 'NEW',
 '9.6',
 '9.3',
 '8.0',
 '6.9',
 '8.8',
 '8.9',
 'No Rating',
 '9.0',
 '9.7',
 '9.4',
 '7.3',
 '9.1',
 '8.5',
 '10',
 '9.9',
 '7.2',
 '7.7',
 '7.5',
 '8.9',
 '7.8',
 '8.6']

In [302]:
Total_Reviews = driver.find_elements_by_xpath("//*[@class='reviews']")
Total = []
for i in Total_Reviews:
    Total.append(i.text)

Total

['6732 Total Reviews',
 '10819 Total Reviews',
 '1072 Total Reviews',
 '206 Total Reviews',
 '248 Total Reviews',
 '443 Total Reviews',
 '130 Total Reviews',
 '513 Total Reviews',
 '0 Total Reviews',
 '8707 Total Reviews',
 '13126 Total Reviews',
 '3935 Total Reviews',
 '391 Total Reviews',
 '4561 Total Reviews',
 '186 Total Reviews',
 '193 Total Reviews',
 '3951 Total Reviews',
 '8109 Total Reviews',
 '879 Total Reviews',
 '3654 Total Reviews',
 '11321 Total Reviews',
 '3402 Total Reviews',
 '3165 Total Reviews',
 '99 Total Reviews',
 '998 Total Reviews',
 '6231 Total Reviews',
 '975 Total Reviews',
 '6324 Total Reviews',
 '3598 Total Reviews',
 '593 Total Reviews']

In [307]:
Overall_Reviews = driver.find_elements_by_xpath("//*[@class='keyword']")
Overall = []
for i in Overall_Reviews[3:]:
    Overall.append(i.text)

Overall

['Very Good',
 'Fabulous',
 'Superb',
 'Rating',
 'Very Good',
 'No Rating',
 'Fabulous',
 'Superb',
 'NEW',
 'Superb',
 'Superb',
 'Fabulous',
 'Good',
 'Fabulous',
 'Fabulous',
 'No Rating',
 'Superb',
 'Superb',
 'Superb',
 'Very Good',
 'Superb',
 'Fabulous',
 'Superb',
 'Superb',
 'Very Good',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Very Good',
 'Fabulous']

In [308]:
#Checking the length of data extracted
print(len(Hostel_Name),len(Distance),len(Rating),len(Total),len(Overall))

30 30 30 30 30


In [310]:
view_button = driver.find_elements_by_xpath("//*[@class='view-button']")

In [323]:
URL = []
for i in driver.find_elements_by_xpath("//*[@class='view-button']"):
    URL.append(i.get_attribute('href'))

URL

['https://www.hostelworld.com/pwa/hosteldetails.php/Generator-London/London/510?from=2021-07-16&to=2021-07-19&guests=1',
 'https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Village/London/502?from=2021-07-16&to=2021-07-19&guests=1',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Safestay-London-Kensington-Holland-Park/London/93920?from=2021-07-16&to=2021-07-19&guests=1',
 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove-The-Crown-Battersea/London/60336?from=2021-07-16&to=2021-07-19&guests=1',
 'https://www.hostelworld.com/pwa/hosteldetails.php/The-London-Home-Hostel/London/80690?from=2021-07-16&to=2021-07-19&guests=1',
 'https://www.hostelworld.com/pwa/hosteldetails.php/247london-Hostel-and-Private-Rooms/London/64523?from=2021-07-16&to=2021-07-19&guests=1',
 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove-The-White-Ferry-Victoria/London/39761?from=2021-07-16&to=2021-07-19&guests=1',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Prime-Bac

In [327]:
Property_desc = []
Facilities = []

for i in URL:
    driver.get(i)
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the Property Desc
    try:
        Property_desc.append(driver.find_element_by_xpath("//*[@class='content collapse-content']").text)
    except NoSuchElementException as e1:
        Property_desc.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    
    #Extracting the Facilities
    try:
        Facilities.append(driver.find_element_by_xpath("//*[@class='facilities']").text)
    except NoSuchElementException as e:
        Facilities.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds
                             

In [328]:
hostel = pd.DataFrame({'Name':Hostel_Name,'Distance':Distance,'Ratings':Rating,'Total_reviews':Total, 'Overall_Reviews':Overall, 'Property_Description':Property_desc,'Facilities':Facilities})
hostel

,Name,Distance,Ratings,Total_reviews,Overall_Reviews,Property_Description,Facilities
0,Generator London,Hostel - 3km from city centre,7.5,6732 Total Reviews,Very Good,Generator London is a design hotel-hostel loca...,Linen Included Free City Maps Free WiFi Free I...
1,St Christopher's Village,Hostel - 1.8km from city centre,8.8,10819 Total Reviews,Fabulous,COVID 19 Policy Update.\nIn response to Corona...,Linen Included Free City Maps Free WiFi Free I...
2,Safestay London Kensington Holland Park,Hostel - 5.9km from city centre,9.6,1072 Total Reviews,Superb,Safestay Holland Park\n\nSafestay Holland Park...,Linen Included Free WiFi
3,"PubLove @ The Crown, Battersea",Hostel - 4.7km from city centre,5.6,206 Total Reviews,Rating,Do London like a local.\nStay just outside the...,Linen Included Free City Maps Free WiFi
4,The London Home Hostel,Hostel - 3.6km from city centre,7.7,248 Total Reviews,Very Good,If you are looking to meet and socialise with ...,Linen Included Free WiFi Free Internet Access
5,247london Hostel and Private Rooms,Hostel - 8.2km from city centre,No Rating,443 Total Reviews,No Rating,247london Hostel and Private Rooms is a home a...,Linen Included Free WiFi Free Internet Access
6,"PubLove @ The White Ferry, Victoria",Hostel - 2.4km from city centre,8.9,130 Total Reviews,Fabulous,"Walk to the river, turn left, and just keep wa...",Linen Included Free City Maps Free WiFi
7,Prime Backpackers Angel,Hostel - 3.6km from city centre,10,513 Total Reviews,Superb,The hostel is housed in a historic building in...,Linen Included Free City Maps Towels Included ...
8,The Dictionary Hostel,Hostel - 4.1km from city centre,NEW,0 Total Reviews,NEW,A friendly hostel situated in the heart of Sho...,Linen Included Free City Maps Free WiFi Free I...
9,Astor Museum Inn,Hostel - 2.2km from city centre,9.6,8707 Total Reviews,Superb,With the British Museum literally on its doors...,Linen Included Free City Maps Free WiFi


In [329]:
#Saving the data into a csv file
hostel.to_csv("Hostel_Details.csv")

#Closing the driver
driver.close()